In [3]:
import time

import numpy as np
from Utils.discrete_family import discrete_family
from Utils.barrier_affine import solve_barrier_tree, solve_barrier_tree_nonneg, solve_barrier_tree_box_PGD
from CART import RegressionTree
from scipy.interpolate import interp1d
from scipy.stats import multivariate_normal as mvn
from Utils.plotting import  *
from Utils.simulation_helpers import *
from scipy.stats import norm as ndist

# For tree-values
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector

# Select a CRAN mirror to download from
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1)  # Select the first mirror

# Install 'remotes' if it's not already installed
if not rpackages.isinstalled('remotes'):
    utils.install_packages(StrVector(('remotes',)))

import rpy2.robjects as ro

from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects import numpy2ri
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


# Main ideas
1. Generate data from the tree-values setting:
> We simulate $X \in \mathbb{R}^{n \times p}$ with $n=200, p=10, X_{i j} \stackrel{i . i . d .}{\sim} N(0,1)$, and $y \sim N_n\left(\mu, \sigma^2 I_n\right)$ with $\sigma=5$ and $\mu_i=b \times\left[1_{\left(x_{i, 1} \leq 0\right)} \times\left\{1+a 1_{\left(x_{i, 2}>0\right)}+1_{\left(x_{i, 3} \times x_{i, 2}>0\right)}\right\}\right]$. 
2. Fix a maximum depth $max\_depth$, e.g. $5$.
3. Use naive inference, tree-values, and our proposed method to fit the tree.
4. Coverage rate comparison: compare the average coverage rate over CIs for terminal nodes in each simulation.
5. Average length comparison: compare the average CIs lengths for terminal nodes in each simulation.
6. Selection quality comparison: 
a. generate the same dataset with same $X, \mu$ but new $\epsilon$,
b. generate new $X, \mu$, \epsilon$ using the same parameters,
compare the MSE of fitted trees on this test set.

In [4]:
# Activate automatic conversion between pandas and R data frames
pandas2ri.activate()

# Import R libraries
treevalues = importr('treevalues')
rpart = importr('rpart')

def tree_values_inference(X, y, mu, sd_y, max_depth=5, level=0.1,
                          X_test=None):
    # Convert the NumPy matrix to an R matrix
    X_r = numpy2ri.py2rpy(X)
    y_r = numpy2ri.py2rpy(y)
    
    # Assign the R matrix to a variable in the R environment (optional)
    ro.globalenv['X_r'] = X_r
    ro.globalenv['y_r'] = y_r
    ro.globalenv['p'] = X.shape[1]
    
    # Construct dataset
    ro.r('data <- cbind(y_r, X_r)')
    # Set the column names to "y", "x1", "x2", ..., "x10"
    ro.r('colnames(data) <- c("y", paste0("x", 1:p))')
    ro.r('data = data.frame(data)')
    
    # Define the rpart tree model
    tree_cmd = 'bls.tree <- rpart(y ~ ., data=data, model = TRUE, cp=0.00, maxdepth=' + str(max_depth) + ')'
    ro.r(tree_cmd)
    bls_tree = ro.r('bls.tree')
    # Plot the tree values (this will plot directly if you have a plotting backend set up)
    #ro.r('treeval.plot(bls.tree, inferenceType=0)')
    
    #ro.r('print(row.names(bls.tree$frame)[bls.tree$frame$var == "<leaf>"])')
    ro.r('leaf_idx <- (row.names(bls.tree$frame)[bls.tree$frame$var == "<leaf>"])')
    leaf_idx = ro.r['leaf_idx']
    
    # Get node mapping
    ro.r('idx_full <- 1:nrow(bls.tree$frame)')
    ro.r('mapped_idx <- idx_full[bls.tree$frame$var == "<leaf>"]')
    
    len = []
    coverage = []
    len_naive = []
    coverage_naive = []
    
    for i,idx in enumerate(leaf_idx):
        # Get the branch information for a specific branch in the tree
        command = 'branch <- getBranch(bls.tree, ' + str(idx) + ')'
        ro.r(command)
        # Perform branch inference
        ro.r('result <- branchInference(bls.tree, branch, type="reg", alpha = 0.10)')
        # Get confidence intervals
        confint = ro.r('result$confint')
        len.append(confint[1] - confint[0])
    
        target_cmd = "contrast <- (bls.tree$where == mapped_idx[" + str(i+1) + "])"
        ro.r(target_cmd)
        contrast = ro.r('contrast')
        contrast = np.array(contrast)
        
        contrast = np.array(contrast * 1 / np.sum(contrast))
        
        target = contrast.dot(mu)
        coverage.append(target >= confint[0] and target <= confint[1])
        
        # Naive after tree value
        # Confidence intervals
        naive_CI = [contrast.dot(y) - 
                    np.linalg.norm(contrast) * sd_y * ndist.ppf(1 - level / 2),
                    contrast.dot(y) + 
                    np.linalg.norm(contrast) * sd_y * ndist.ppf(1 - level / 2)]
        coverage_naive.append((target >= naive_CI[0] and target <= naive_CI[1]))
        len_naive.append(naive_CI[1] - naive_CI[0])
    
    if X_test is not None:
        X_test_r = numpy2ri.py2rpy(X_test)
        ro.globalenv['X_test_r'] = X_test_r
        ro.r('pred <- predict(bls.tree, data = X_test_r)')
        pred = ro.r['pred']
    else:
        pred = None
        
    return (np.mean(coverage), np.mean(len), 
            np.mean(coverage_naive), np.mean(len_naive), pred) 

In [5]:
def generate_test(mu, sd_y):
    n = mu.shape[0]
    return mu + np.random.normal(size=(n,), scale=sd_y)

In [6]:
def randomized_inference(reg_tree, sd_y, y, mu, level=0.1):
    #print(reg_tree.terminal_nodes)
    coverage_i = []
    lengths_i = []
    
    for node in reg_tree.terminal_nodes:
        pval, dist, contrast, norm_contrast, obs_tar, logW, suff, sel_probs \
            = (reg_tree.condl_node_inference(node=node, 
                                             ngrid=10000,
                                             ncoarse=200,
                                             grid_w_const=1.5,
                                             reduced_dim=1,
                                             sd=sd_y,
                                             use_cvxpy=True))
        target = contrast.dot(mu)
        
        # This is an interval for
        # eta_*'mu = eta'mu / (norm(eta) * sd_y)
        selective_CI = (dist.equal_tailed_interval(observed=norm_contrast.dot(y),
                                                   alpha=level))
        selective_CI = np.array(selective_CI)
        selective_CI *= np.linalg.norm(contrast) * sd_y
        coverage_i.append((target >= selective_CI[0] and target <= selective_CI[1]))
        lengths_i.append(selective_CI[1] - selective_CI[0])
        
    return coverage_i, lengths_i

In [7]:
def terminal_inference_sim(n = 50, p = 5, a = 0.1, b = 0.1, 
                           sd_y = 1, noise_sd_list = [0.5, 1, 2, 5], 
                           start=0, end=100,
                           level=0.1):
    coverage_dict = {m: [] for m in noise_sd_list + ["Tree val", "Naive"]}
    length_dict = {m: [] for m in noise_sd_list + ["Tree val", "Naive"]}
    MSE_dict = {m: [] for m in noise_sd_list + ["Tree val", "Naive"]}
    
    for i in range(start, end):
        print(i, "th simulation")
        np.random.seed(i + 48105)
        X = np.random.normal(size=(n, p))

        mu = b * ((X[:, 0] <= 0) * (1 + a * (X[:, 1] > 0) + (X[:, 2] * X[:, 1] <= 0)))
        y = mu + np.random.normal(size=(n,), scale=sd_y)
        y_test = generate_test(mu, sd_y)
        
        for noise_sd in noise_sd_list:
            # Create and train the regression tree
            reg_tree = RegressionTree(min_samples_split=10, max_depth=3,
                                      min_proportion=0.1)
            reg_tree.fit(X, y, sd = noise_sd * sd_y)
            
            coverage_i, lengths_i = randomized_inference(reg_tree=reg_tree, 
                                                         y=y, sd_y=sd_y, mu=mu, level=level)
            pred_test = reg_tree.predict(X)
            MSE_test = (np.mean((y_test - pred_test) ** 2))
            # Record results
            coverage_dict[noise_sd].append(np.mean(coverage_i))
            length_dict[noise_sd].append(np.mean(lengths_i))
            MSE_dict[noise_sd].append(MSE_test)
        
        # Tree value & naive inference & prediction
        (coverage_treeval, avg_len_treeval, 
         coverage_treeval_naive, avg_len_treeval_naive,
         pred_test_treeval) = tree_values_inference(X, y, mu, 3, X_test=X, max_depth=3)

        MSE_test_treeval = (np.mean((y_test - pred_test_treeval) ** 2))
        coverage_dict["Tree val"].append(coverage_treeval)
        length_dict["Tree val"].append(avg_len_treeval)
        MSE_dict["Tree val"].append(MSE_test_treeval)
        coverage_dict["Naive"].append(coverage_treeval_naive)
        length_dict["Naive"].append(avg_len_treeval_naive)
        MSE_dict["Naive"].append(MSE_test_treeval)
        
    return coverage_dict, length_dict, MSE_dict

In [9]:
(coverage_dict, length_dict, MSE_dict) \
    = terminal_inference_sim(start=0, end=5, n=50, sd_y=1, noise_sd_list=[0.5, 1], 
                             a=1,b=1, level=0.1)

0 th simulation
1 th simulation
2 th simulation
3 th simulation
4 th simulation


In [10]:
coverage_dict, length_dict, MSE_dict

({0.5: [np.float64(1.0),
   np.float64(0.8333333333333334),
   np.float64(1.0),
   np.float64(0.8571428571428571),
   np.float64(1.0)],
  1: [np.float64(1.0),
   np.float64(1.0),
   np.float64(1.0),
   np.float64(0.8571428571428571),
   np.float64(1.0)],
  'Tree val': [np.float64(1.0),
   np.float64(1.0),
   np.float64(0.75),
   np.float64(1.0),
   np.float64(1.0)],
  'Naive': [np.float64(0.5),
   np.float64(0.5),
   np.float64(0.25),
   np.float64(0.25),
   np.float64(0.4)]},
 {0.5: [np.float64(90.83689452040015),
   np.float64(125.74149804162376),
   np.float64(42.8361530412838),
   np.float64(150.94684204209943),
   np.float64(247.37451939391073)],
  1: [np.float64(102.6563796029573),
   np.float64(55.97499486748189),
   np.float64(51.67793810368926),
   np.float64(75.93056926570041),
   np.float64(258.2736888922537)],
  'Tree val': [np.float64(26.015825722513924),
   np.float64(8.709016787522842),
   np.float64(76.60651168458173),
   np.float64(15.399370550237458),
   np.float64(29

In [28]:
noise_sd_list = [0.5, 1, 2, 5] + ["Tree val", "Naive"]

In [29]:
noise_sd_list

[0.5, 1, 2, 5, 'Tree val', 'Naive']